# SymPy

You can do Mathematica-style symbolic algebra in the same environment as your data analysis.

In [ ]:
!pip install sympy --user

In [ ]:
import sympy
sympy.init_printing()

In [ ]:
E1, px1, py1, pz1, eta1, phi1 = sympy.symbols("E1, px1, py1, pz1, eta1, phi1", real=True)
E2, px2, py2, pz2, eta2, phi2 = sympy.symbols("E2, px2, py2, pz2, eta2, phi2", real=True)
pt1, pt2 = sympy.symbols("pt1, pt2", nonnegative=True)

In [ ]:
pt1eq = sympy.Eq(pt1, sympy.sqrt(px1**2 + py1**2))
pt2eq = sympy.Eq(pt2, sympy.sqrt(px2**2 + py2**2))
pt1eq

In [ ]:
eta1eq = sympy.Eq(eta1, sympy.atanh(pz1 / sympy.sqrt(px1**2 + py1**2 + pz1**2)))
eta2eq = sympy.Eq(eta2, sympy.atanh(pz2 / sympy.sqrt(px2**2 + py2**2 + pz2**2)))
eta1eq

In [ ]:
phi1eq = sympy.Eq(phi1, sympy.atan2(py1, px1))
phi2eq = sympy.Eq(phi2, sympy.atan2(py2, px2))
phi1eq

Invert some non-linear identities.

In [ ]:
sympy.solve([pt1eq, eta1eq, phi1eq], px1, py1, pz1)

It's returning three solutions and is too pedantic about signs. I'll just read off the right transformations.

In [ ]:
mass = sympy.sqrt((E1 + E2)**2 - (px1 + px2)**2 - (py1 + py2)**2 - (pz1 + pz2)**2)
mass

In [ ]:
mass2 = sympy.trigsimp(mass.subs(px1, pt1*sympy.cos(phi1))
                           .subs(py1, pt1*sympy.sin(phi1))
                           .subs(pz1, pt1*sympy.sinh(eta1))
                           .subs(px2, pt2*sympy.cos(phi2))
                           .subs(py2, pt2*sympy.sin(phi2))
                           .subs(pz2, pt2*sympy.sinh(eta2)))
mass2

A nice benefit of doing symbolic math in Python is that you can convert symbolic expressions into functions for your data analysis.

In [ ]:
f = sympy.lambdify([E1, pt1, eta1, phi1, E2, pt2, eta2, phi2], mass2, modules=["numpy"])

`f` is a function you can execute (evaluating as "`numpy`" rather than the default "`math`").

In [ ]:
f(10, 8, 0, 0, 9, 7, 0, 0)

# iminuit

iminuit lets you fit functions in Python. It's a clone of PyMinuit, which I created to fit the data in my thesis.  `:)`

The installation takes a minute or two— it's probably compiling MINUIT.

In [ ]:
!pip install iminuit --user

In [ ]:
from iminuit import Minuit

In [ ]:
def f(x, y, z):
    return (x - 2)**2 + (y - 3)**2 + (z - 4)**2

minuit = Minuit(f, x=0, y=0, z=0, error_x=1, error_y=1, error_z=1, errordef=1)
minuit.migrad()
minuit.minos()
print(minuit.values)
print(minuit.errors)
print(minuit.get_merrors())

There are also projects built on iminuit (it's becoming an ecosystem).

In [ ]:
!pip install probfit --user

In [ ]:
import numpy
import probfit

data = numpy.random.normal(0, 1, 10000)
unbinned_likelihood = probfit.UnbinnedLH(probfit.gaussian, data)
minuit = Minuit(unbinned_likelihood, mean=0.1, sigma=1.1)
minuit.migrad()

In [ ]:
!pip install matplotlib==2.0.2 --user

In [ ]:
plot = unbinned_likelihood.draw(minuit)

In [ ]:
plot = minuit.draw_mnprofile("mean")

In [ ]:
plot = minuit.draw_mncontour("mean", "sigma")

# NumPythia

(I mostly like the name.)

In [ ]:
!pip install numpythia --user

In [ ]:
import numpythia
import numpythia.testcmnd
import pandas

pythia = numpythia.Pythia(numpythia.testcmnd.get_cmnd("w"), random_state=1)

In [ ]:
events = list(pythia(events=1))    # pythia is a generator; evaluate it
pandas.DataFrame(events[0].all())  # event.all(SELECTION) returns a Numpy record array, so of course we Pandas it

# pyjet

FastJet in Numpy

In [ ]:
!pip install pyjet --user

In [ ]:
import pyjet

fourvectors = events[0].all()[["E", "px", "py", "pz"]]
for jet in pyjet.cluster(fourvectors, R=1.0, p=-1, ep=True).inclusive_jets():
    print(jet)

# PyPDT

In [ ]:
!pip install pypdt --user

In [ ]:
import pypdt

for pdgid in set(events[0].all()["pdgid"]):
    p = pypdt.get(pdgid)
    if p is not None:
        print("{}: {} mass: {} width {} lifetime {} ctau {}".format(
            pdgid, p.name, p.mass, p.width, p.lifetime, p.ctau))